In this blog post, we are going to use Spotify Web API to help us get the audio features (i.e. danceablity, loudness,etc.) of an user-specified song on Spotify. These audio features are helpful to genre identification.

In [8]:
import spotipy
import spotipy.oauth2
import requests
import pandas as pd

First we’ll use our Client ID and Client Secret to generate an access token so we can pull data from Spotify’s API. I figured out how to do this using the help of this [blog post](https://stmorse.github.io/journal/spotify-api.html).

We define a function for repeated use of getting track features for a certain audio. The function requires the track_id of a song from the user input. According to [Web API docs](https://developer.spotify.com/documentation/web-api/) and this blog post, we can get track id by:
1. Click to the three dots next to the artist name/album/track title
2. Click 'Share'
3. Click 'Copy Song Link' from the second menu
4. The id is at the very end of the URL; that is, the URL is in the form of "https://open.spotify.com/track/"+id"

In [9]:
def get_track_features(track_id):
    """
    Access the input of a song's Spotify ID,
    output the features in a dataframe
    """
    # use Client ID and Client Secret for Spotify Developer
    CLIENT_ID = "8e256a7b32f845ec9aebf81ae0feadb4"
    CLIENT_SECRET = '928d1a35600041eea6c69f97205fac6c'
    
    AUTH_URL = 'https://accounts.spotify.com/api/token'

    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()

    # save the access token
    access_token = auth_response_data['access_token']
    
    headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)
    }
    
    # base URL of all Spotify API endpoints
    BASE_URL = 'https://api.spotify.com/v1/'

    # Track ID from the URI
    track_id = track_id

    # actual GET request with proper header
    r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
    r = r.json()
    
    # r is a dictionary with our desired feature and values
    # transform the dictionary into a dataframe
    # drop the unnecessary columns
    features = pd.DataFrame(r,index = [0])
    features = features.drop(["uri","track_href","analysis_url"],axis = 1)
    
    return features

In [10]:
get_track_features("75MNhvTCCKsST3YqqUiU9r?si=0d2972b96c474633")

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,duration_ms,time_signature
0,0.788,0.859,2,-2.724,1,0.0856,0.281,0,0.0424,0.822,141.02,audio_features,75MNhvTCCKsST3YqqUiU9r,207853,4
